# Module 3: Factor analysis

In this lab you will learn about **Factor Analysis** (FA),
which is a linear factor model that not only assumes observables are a linear combination of factors
(or latent variables) plus noise, but also that they follow Gaussian distribution.
In addition, observed variables are assumed to be conditionally independent, given latent variables.

Some key aspects to focus on:
+ Fewer factors than original features in data space.
+ Different types of methods and solutions.
+ More elaborate framework than principal component analysis.

For this session, we are going to use **red wine quality** dataset for factor analysis and
transform its data space into feature space with 5 factors.

sklearn API reference:

+ [sklearn.decomposition.FactorAnalysis](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FactorAnalysis.html)

**Note:** Previously you saw Factor Analysis in the 8610 (Stat/Math) course.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import numpy as np
import pandas as pd
from sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import scale


## Load dataset

In [ ]:
# Dataset location
DATASET = '/dsa/data/all_datasets/wine-quality/winequality-red.csv'
assert os.path.exists(DATASET)

# Load and shuffle
dataset = pd.read_csv(DATASET, sep=';').sample(frac = 1).reset_index(drop=True)

X = np.array(dataset.iloc[:, :-1])
y = np.array(dataset.quality)

dataset.describe()

In [ ]:
dataset.head()

## Factor analysis with sklearn

In [ ]:
fa = FactorAnalysis(n_components=5)
X_features = fa.fit_transform(X)
print('Features shape', X_features.shape)

## Estimation of the factor model

Factor analysis essentially proposes the following to explain the structure of the observables:

$$ X - \mu = LF + \varepsilon $$

+ L: Factor loadings
+ F: Features

That is to assume observables are a linear combination of latent variables plus noise.
And this is an estimation problem that usually takes some numerical computation to solve iteratively.

In practice, to solve factor analysis, the **goal** would be to find estimates of factor loadings **L** and specific variances **Ψ**, such that:

$$ cov(X) = LL^T + cov(\varepsilon) = LL^T + \Psi $$


### Factor loadings

The factor loadings is the matrix **L** would take latent variables and transform them to observables **X** minus its mean and noise.
The following cell prints one found by `sklearn`.

In [ ]:
def FactorLoadings(components, n_components=5):
    """This functions puts a frame on the loadings matrix for prettified printing"""
    return pd.DataFrame(components.T,
        columns = ['Factor {}'.format(i+1) for i in range(n_components)],
        index = dataset.columns[:-1])

FactorLoadings(fa.components_, n_components=5)

### Specific variances

The specific variances matrix **Ψ** is a _diagonal matrix_ representing the variances of 
noise in the model with the following elements on the diagonal.

In [ ]:
fa.noise_variance_

## Reconstruction of data space

The factor analysis models the observables **X** as a linear combination of factors plus noise.
Therefore, it should be insteresting to reconstruct data space with some solution appropiate for the formulation of factor analysis.

In [ ]:
print('Factors shape', fa.components_.shape)
noise = np.random.multivariate_normal(np.mean(X, axis = 0), np.diag(fa.noise_variance_), X.shape[0])
X_reconstructed = np.dot(X_features, fa.components_) + noise
print('Reconstructed dataset shape', X_reconstructed.shape)

The following cell shows a reconstructed dataset which is an approximation of original dataset.

In [ ]:
pd.DataFrame(X_reconstructed, columns = dataset.columns[:-1]).head()

The following cell shows the original dataset for comparison.
They **could appear to be very different**, although still following some general pattern,
because random noise was just introduced,
but this aims to demonstrate the connection between data space and feature space.

In [ ]:
dataset.iloc[:5, :-1]

## Verify covariance structure of FA

A more meaningful thing to try than restructing data space could be verifying its covariance structure,
because unlike the previous example, we don't have to re-introduce noise which was lost during factor analysis.

In practice, a proper solution to FA is usually verfied by plugging into the claim 1 below.

### Claim 1:

$$ cov(X) = LL^T + \Psi $$

+ L: Factor loadings
+ Ψ: Specific variance / noise variance

In [ ]:
X_centered = scale(X, with_std = False)

print(np.allclose(
    np.dot(X_centered.T, X_centered) / X.shape[0],                # left hand side: covariance matrix of X
    np.dot(fa.components_.T, fa.components_) + np.diag(fa.noise_variance_)   # right hand side
))

print(np.isclose(
    np.dot(X_centered.T, X_centered) / X.shape[0],                # left hand side: covariance matrix of X
    np.dot(fa.components_.T, fa.components_) + np.diag(fa.noise_variance_),  # right hand side
atol=1e-2, rtol=1e-1).astype('int'))

Although this factor analysis solution provided by sklearn wasn't very precise, it's still close and useful.

The following cell compares our calculations with those provided by numpy and sklearn packages.
It turns out that the computation of **left_hand_side** and **right_hand_side** are accurate to definition.

In [ ]:
print(np.allclose(
    np.dot(X_centered.T, X_centered) / X.shape[0],                # left hand side: covariance matrix of X
    np.cov(X, rowvar = False, bias = True)                        # covariance matrix by numpy
))

print(np.allclose(
    np.dot(fa.components_.T, fa.components_) + np.diag(fa.noise_variance_),  # right hand side
    fa.get_covariance()                                           # covariance matrix by sklearn
))

### Claim 2:

$$ cov(X, F) = L$$ 

+ L: Factor loadings
+ F: Features

In [ ]:
print(np.allclose(
    np.mean(X[..., np.newaxis] * np.expand_dims(X_features, 1), axis = 0), # left hand side
    fa.components_.T,                                                      # right hand side
rtol = 1e-3))

## The principal component solution

Factor analysis provides a latent linear model to explain the observables **x**, 
which serves as a valuable insight towards feature extraction.
However, different types of methods have been proposed for solving factor analysis, including the **principal component method** and the **maximum likelihood method**.

Now, we plug principal components into covariance structure of factor analysis
to verify this is one of its solutions. 
This also demonstrates the relation between FA and PCA.

### PCA transform

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
PCA_features = pca.fit_transform(X_centered)

### Verify claim 1

$$cov(X) = L L^T + \Psi$$

+ L: Factor loadings - provided by spectral decomposition
+ Ψ: Noise variance - set to 0 becaues PCA doesn't model noise

In [ ]:
print(np.isclose(
    np.dot(X_centered.T, X_centered) / X.shape[0],      # left hand side: covariance matrix of X
    sum(eigenvalue*np.outer(eigenvector,eigenvector)    # right hand side: spectral decomposition
        for eigenvalue, eigenvector in zip(pca.explained_variance_, pca.components_)),
atol=1e-2, rtol=1e-1).astype('int'))

Note: PCA could be considered a special case to FA.

## Factor loadings

The following is the factor loadings offered by PCA. This is a corollary from spectral decomposition in its additive form. 
Please refer to [_spectral decomposition_](https://en.wikipedia.org/wiki/Eigendecomposition_of_a_matrix) for further information if interested.

$$cov(X)=\sum \lambda_i q_i q_i^T = LL^T $$

In [ ]:
FactorLoadings(np.vstack([np.sqrt(eigenvalue)*eigenvector
    for eigenvalue, eigenvector in zip(pca.explained_variance_, pca.components_)]))

### Factor analysis vs principal component analysis

+ FA imposes a structure with fixed number of factors; PCA analyzes the eigenstructure of data and provides principal components in descreasing order of importance.
+ FA focuses on interpretation of data; PCA focuses on maximizing variances.
+ FA provides a model that needs estimation techniques to solve; PCA is a well-defined algorithm with unique solution.
+ FA and PCA both assume the linear structure of the data and utilize similar set of mathematical tools.

## Scree plot

This plots variances (y-axis) against components (x-axis).
It helps us to decide on number of principal components to be retained, although this is subjective.

In [ ]:
explained_variance = np.flip(np.sort(np.sum(fa.components_**2, axis=1)), axis=0)
x_ticks = np.arange(len(fa.components_))+1
plt.xticks(x_ticks) # this enforces integers on the x-axis
plt.plot(x_ticks, explained_variance)

Similarily, you could also plot the explained variance ratio.
FA should not have total explained variance ratio equal to 1 because of noise variance.

In [ ]:
explained_variance_ratio = explained_variance/(np.sum(explained_variance)+np.sum(fa.noise_variance_))
plt.xticks(x_ticks) # this enforces integers on the x-axis
plt.plot(x_ticks, explained_variance_ratio)
print('total expained variance ratio', np.sum(explained_variance_ratio))

## Conclusion

In this lab we learned about:
+ Apply FA to a dataset for feature extraction.
+ Reconstruction of original dataset from FA.
+ Convariance structure of FA.
+ Comparing FA and PCA.
+ Scree plot of FA.